# Tutorial: Use a trained RNN in grainLearning calibration process.

## ⚙️ Install grainlearning package
(Not necessary if you are running jupyter-lab on an environment where grainlearning and rnn dependencies are installed)

```bash 
pip install grainlearning --extras "rnn"
```

## Introduction to this tutorial
We hope that now you are familiar with grainLearning and the RNN module. In this tutorial we are going to explore how can we use a pretrained neural network as a surrogate model in the grainLearning calibration process. In such context, the RNN plays the role of a `DynamicSystem`.

We consider the case of *Drained Triaxial Compression*, and have two resources:

1. A RNN trained model on several DEM simulations of *Drained Triaxial Compression* tests. 
2. The experimental measurements of a *Drained Triaxial Compression* test.

The *objective* is to find which set of `contact_params` of the DEM simulation would give us an equivalent to our real-world material.

This tutorial has three main parts:

1. Prepare the pre-trained model and observation data.
2. Create a callback function to link to `DynamicSystem`.
3. GrainLearning calibration loop.

Let's start importing some useful packages:

In [2]:
import numpy as np
import tensorflow as tf

from grainlearning import BayesianCalibration
import grainlearning.dynamic_systems
import grainlearning.rnn.predict as predict_rnn
import grainlearning.rnn.preprocessing as preprocessing_rnn

## 1.a Prepare the pre-trained model 🗄️
For the purpose of this tutorial we are going to take an example from the a dataset containing DEM simulation results of a triaxial compression. 
Our RNN model was trained in a similar database, and we are going to take one example to infer from. In practice, such example could be the results of real-world experiments for wich we do not know the DEM _contact parameters_.

In [3]:
path_to_trained_model = '/Users/luisaorozco/Documents/Projects/GrainLearning/grainLearning/grainlearning/rnn/trained_models/rnn_triaxial_drained'

In [4]:
# Load the pre-trained model
model_rnn, train_stats, config_rnn = predict_rnn.get_pretrained_model(path_to_trained_model)

Metal device set to: Apple M1 Pro

systemMemory: 32.00 GB
maxCacheSize: 10.67 GB



2023-03-23 16:31:23.241391: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-03-23 16:31:23.241521: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


## 1.b Prepare the observation data 🥼
We load the data that we have obtained in an experiment (drained triaxial comrpession) and for which we want to calibrate contact parameters to use in DEM.
In the first column there is the time-sequence of the control parameter $\varepsilon_{axial}$, and the 3 next columns are the observed parameters $e,\ p,\ q$.

In [5]:
path_to_data = 'experimental_test_drained_s3=0.2.dat'

In [6]:
experiment_data = np.loadtxt(path_to_data) 
inputs = experiment_data[:,0]
outputs = experiment_data[:,1:]

`extra_contact_params` are not in `system.param_data` i.e. they are not parameters that need to be inferred. 
However, these are control parameters necessary to predict from RNN that are added at the end of  `contact_params`.
Given the values of `add_e0`, `add_pressure` and `add_experiment_type` in `config`, we can determine how many parmeters are extra.

In [7]:
extra_contact_params = [0.2] # confining pressure

If no padding was taking into account during training, the predicitions of the RNN are going to be one `window_size` shorter.
We adapt the labels so that during the calibration we compare tensors with the same dimensions.

In [8]:
if 'pad_length' not in config_rnn or config_rnn['pad_length'] == 0: # no padding
    outputs = outputs[config_rnn['window_size']:, :]

In Grainlearning, the temporal dimension is always at the end, we need to switch the access of the data to comply with this format.

In [9]:
# convert arrays from [num_time_steps, num_features] -> [num_features, num_time_steps]
outputs = np.moveaxis(outputs, 0, -1)

In this preparation steps we got few important arrays to work with:
- `inputs`: control of the system ($\varepsilon_{axial}$)
- `outputs`: observation experimental data ($e,\ p,\ q$)
- `extra_contact_params`: Additional control parameters that don't need to be inferred but need ot be passed along with `contact_params` to the RNN.
- `config_rnn`: dictionary containing the configration of the pre-trained RNN model.

## 2. Create a callback function to link to `DynamicSystem`

In [10]:
def predict_with_RNN(system, **_):
    window_size = config_rnn['window_size']
    sequence_length = np.shape(system.ctrl_data)[0]
    
    # For compatibility with the RNN (first dimension is sample)
    load_sequence = np.expand_dims(inputs, axis=0) # only needed if system.ctrl_data ahs a single dimension
    load_sequence = np.moveaxis(load_sequence, 0, -1)
    load_sequence = np.repeat(load_sequence[np.newaxis, :], system.num_samples, axis=0)
    
    # add extra_contact_params to the contact_params used to draw a prediction
    contact_params = np.array([np.append(i, extra_contact_params) for i in system.param_data])
    
    data_inputs = ({'load_sequence': load_sequence, 'contact_parameters': contact_params}, load_sequence)
    data_inputs = tf.data.Dataset.from_tensor_slices(data_inputs)
    predictions = predict_rnn.predict_macroscopics(model_rnn, 
                                                   data_inputs, 
                                                   train_stats, 
                                                   config_rnn, 
                                                   batch_size=system.num_samples)
    # Getting rid of extra predictions of the rnn. 
    # We are only interested on the first 3 that we can compare against the observation data.
    predictions = predictions.numpy()[:, :, :3]

    # converting the predictions to GL format (temporal dimension at the end)
    predictions = np.moveaxis(predictions, 1, -1)
    # update sim_data in system
    system.set_sim_data(predictions)

## 3. GrainLearning calibration loop 🔁

In [11]:
def grainlearning_calibration(inputs: np.array, outputs: np.array):
    """
    Main function defining and driving the grainLearning calibration.
    1. Define the BayesianCalibration and all its elements: DynamicSystem, inferences and sampling parameters.
    2. Run the calibration and finally return the inferred parameters.

    :param inputs: numpy.array with the control data.
      In this example (drained triaxial compression) a time sequence of the axial strain.
    :param outputs: numpy.array with the observation data.
      In this example a time sequence with macroscopic observables such as e, p, q.

    :return: most_prob_params numpy.array containing the inferred values of the contact parameters.
    """
    curr_iter = 0
    calibration = grainlearning.BayesianCalibration.from_dict(
        {
            "curr_iter": curr_iter,
            "num_iter": 20,
            "system": {
                "system_type": grainlearning.dynamic_systems.DynamicSystem, # because I'm not reading files, my data is generated by an RNN.
                "obs_data": outputs,    # experimental data
                "obs_names": ['e', 'p', 'q'],
                "ctrl_data": inputs[config_rnn['window_size']:], # Synthetic data.
                "ctrl_name": ['e_z'],  # Only one of the strains (axial)
                "num_samples": 50,     # num of samples (gaussian mixture) generated per iteration
                "sim_name": 'Triaxial compression with RNN, from experiment.',
                # to get these labels I opened the hdf5 file and query: file.attrs['contact_params']
                "param_names": ['E', 'v', 'kr', 'eta', 'mu'],
                "param_min": [7.00195859, 1.6e-4, 8.3e-4, 1.5e-4, 4.5e-2], # using the range of contact params used to train the RNN.
                "param_max": [9.99607979, 0.49952, 0.99875, 0.99878, 59.955],
                "callback": predict_with_RNN
            },
            "calibration": {
                "inference": {"ess_target": 0.3},
                "sampling": {
                "max_num_components": 5,
                "prior_weight": 0.01,
                },
            },
            "save_fig": -1, # Not generating plots, but look at the end of this notebook for more info.
        }
    )

    print("Run the calibration")
    calibration.run()
    print("Calibration finished")
    return calibration.get_most_prob_params()


## 🎛️ Start the calibration

In [15]:
# Run the calibration
most_prob_params = grainlearning_calibration(inputs, outputs)

Run the calibration
Bayesian calibration iter No. 0
Bayesian calibration iter No. 1
Bayesian calibration iter No. 2
Bayesian calibration iter No. 3
Bayesian calibration iter No. 4
Bayesian calibration iter No. 5
Bayesian calibration iter No. 6
Bayesian calibration iter No. 7
Bayesian calibration iter No. 8
Bayesian calibration iter No. 9
Bayesian calibration iter No. 10
Bayesian calibration iter No. 11
Bayesian calibration iter No. 12
Bayesian calibration iter No. 13
Bayesian calibration iter No. 14
Bayesian calibration iter No. 15
Bayesian calibration iter No. 16
Bayesian calibration iter No. 17
Bayesian calibration iter No. 18
Bayesian calibration iter No. 19
Calibration finished


## ⚖️ Compare the results
For comparison purposes, we kne beforehand, what would be the best `contact_params` for our experimental data. We are now going to compare those to the inferred ones (via grainlearning calibration process).

In [16]:
contact_params_reference = [7.57811981, 0.4448, 0.47813, 0.22497, 56.0784]
print(f"Contact params (ground truth): {contact_params_reference} \nContact parameters calibrated via GL: {most_prob_params}")

Contact params (ground truth): [7.57811981, 0.4448, 0.47813, 0.22497, 56.0784] 
Contact parameters calibrated via GL: [ 7.59705764  0.47406657  0.28572617  0.23129764 56.9195252 ]


In [17]:
# get the percentage error
print(f"error = {np.abs(most_prob_params - contact_params_reference) / contact_params_reference}")

error = [0.00249901 0.06579715 0.40240904 0.02812659 0.01499909]


### 📉 Plotting
- Try with `save_fig` equals to 1: the plots for each one of the calibration iterations will be saved in a folder.
- or try with `save_fig` equals to 0: the plots will be printed in this jupyter notebook on the calibration run.

## ✏️ Final tips

- Check always the dimensions and the order of the parameters in the tensors that you are using. 
- Keep in mind that in GrainLearning the last dimension is the time.